## Data Description
The dataset we are using is comprised of 2.5 years of insulin dosage, blood glucose (bg), and Estimated Variability of Glucose (EVG). This data was collected from a type 1 diabetic's insulin pump and continuous glucose monitor (CGM). Unfortunately this data isnt entirely continous over the 2.5 year span and has gaps in some instances. This data has been exported directly from 30 day span .csv files containing 3 tables one for insulin dosage (bolus), bg, and EVG data. We split these files into the 3 respective tables and saved them to new tables containing the full time-span of data, these files are titles Bolus.csv, BG.csv, and EVG.csv. Here I will detail the contents of these files.

1. Bolus
    * Features:
        * Type: Type of bolus event (Always 'Bolus').
        * BolusType: Describes the wayin which the Bolus was used (categorical).
        * BolusDeliveryMethod: Method used to deliver the bolus (Auto or Standard)(categorical).
        * BG (mg/dL): Blood glucose levels at the time of bolus administration (continuous data).
        * SerialNumber: Device identifier.
        * CompletionDateTime: Timestamp when the dosing was completed.
        * InsulinDelivered: The standard unit measure for the amount of insulin delivered (continuous data).
        * FoodDelivered: Insulin delivered for food consumption (continuous data).
        * CorrectionDelivered: Insulin delivered for BG correction (continuous data).
        * CompletionStatusDesc: Status description of the dosage (categorical).
        * BolexStartDateTime: Not Used In Export.
        * BolexCompletionDateTime: Not Used In Export.
        * BolexInsulinDelivered: Not Used In Export.
        * BolexCompletionStatusDesc: Not Used In Export.
        * StandardPercent: (Always 100).
        * Duration (mins):(Always 0).
        * CarbSize: The amount of carbohydrates consumed in grams (continuous data).
        * TargetBG (mg/dL): Target blood glucose level for the subject in milligrams per deciliter (continuous data).
        * CorrectionFactor: Insulin sensitivity factor (continuous data).
        * CarbRatio: Insulin-to-carbohydrate ratio (continuous data).
    * Notes:
        * This table holds the most detailed records for insulin dosing decisions, food intake, and blood glucose corrections.
        * This table will be very useful in modeling relationships between carbohydrate intake, insulin dosage, and BG levels.
        * Bolus (Def: A large single doseage of insuline to lower a bloodsugar rise) refers to the device/method of insuline delivery. Which is automated through a pocket size device with a refillable tank of insulin.
2. EVG Table
    * Features:
        * DeviceType: Type/Name of the device used to record the event.
        * SerialNumber: Device identifier.
        * Description: A text description of the type of data recorded (Always EVG).
        * EventDateTime: Timestamp of when the measurement was recorded.
        * Readings (mg/dL): The estimated glucose level at the recorded time, in milligrams per deciliter (mg/dL).
    * Notes: 
        * The EVG data is collected directly from a CGM.
        * EVG is designed for tracking overall trends and patterns in glucose levels rather than moment-to-moment decisions.
        * Useful for identifying time-in-range, glucose variability, and predicting hypo/hyperglycemia over time.
        
3. BG Table
    * Features:
        * DeviceType: Type/Name of device used to measure blood glucose.
        * SerialNumber: Device identifier.
        * Description: A text description of the type of data recorded (Always BG)
        * EventDateTime: Timestamp of the blood glucose measurement.
        * BG (mg/dL): Blood glucose levels in milligrams per deciliter (continuous data).
        * Note: Additional notes field (Always Blank).
    * Notes:
        * This data is manually entered and is recorded through a glucometer (Finger Prick).
        * The BG measurement reflects the actual glucose levels at the time of measurement.


### General Dataset Properties

* Total \# of Features:
    * Bolus: 18 features
    * EVG: 5 features
    * BG: 6 features
* \# of Usable/Useful Features: (!!!!! Update possibly)
    * Bolus: 12 features
    * EVG: 2 features
    * BG: 2 features
* Table Sizes:
    * Bolus: 11,348 Records
    * EVG: 191,781 Records
    * BG: 2,832 Records
* Unique Dates: 
    * Bolus: 599 Days
    * EVG: 666 Days
    * BG: 663 Days
* Intersecting Dates: 599




### Data Notes
* Not all Days are covered, at points there are significant (5-8 week) gaps in data.
* Different Devices used to collect data may overlap times (device marked as 'UNKNOWN' in data)
* 3 different tables (I think our main focus should be on the third table that has the pump data coupled with the best of the BG tables (1 or 2))

In [2]:
import pandas as pd
import csv
import os


def load_export_csv(file_path):
    # Define the headers for each table to identify them
    table_headers = {
        'table1': ["DeviceType", "SerialNumber", "Description", "EventDateTime", "Readings (mg/dL)"],
        'table2': ["DeviceType", "SerialNumber", "Description", "EventDateTime", "BG (mg/dL)", "Note"],
        'table3': ["Type", "BolusType", "BolusDeliveryMethod", "BG (mg/dL)", "SerialNumber",
                   "CompletionDateTime", "InsulinDelivered", "FoodDelivered", "CorrectionDelivered",
                   "CompletionStatusDesc", "BolexStartDateTime", "BolexCompletionDateTime",
                   "BolexInsulinDelivered", "BolexCompletionStatusDesc", "StandardPercent",
                   "Duration (mins)", "CarbSize", "TargetBG (mg/dL)", "CorrectionFactor",
                   "CarbRatio"]
    }
    
    # Initialize data storage for each table
    data_tables = {
        'table1': [],
        'table2': [],
        'table3': []
    }
    
    current_table = None  # To keep track of which table we're currently reading
    line_number = 0  # To track line numbers for debugging
    
    with open(file_path, 'r', newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            line_number += 1
            # Strip whitespace from each cell
            row = [cell.strip() for cell in row]
            
            # Debugging: Print current row and line number
            # print(f"Line {line_number}: {row}")
            
            # Check if the row matches any table header
            if row[:len(table_headers['table1'])] == table_headers['table1']:
                current_table = 'table1'
                # print(f"Detected header for table1 at line {line_number}")
                continue  # Skip the header row
            elif row[:len(table_headers['table2'])] == table_headers['table2']:
                current_table = 'table2'
                # print(f"Detected header for table2 at line {line_number}")
                continue  # Skip the header row
            elif row[:len(table_headers['table3'])] == table_headers['table3']:
                current_table = 'table3'
                # print(f"Detected header for table3 at line {line_number}")
                continue  # Skip the header row
            elif not any(cell for cell in row):
                # Empty row signifies possible separation; skip
                current_table = None
                # print(f"Detected empty row at line {line_number}; resetting current_table")
                continue
            
            # If current_table is set, append the row to the corresponding data list
            if current_table:
                expected_columns = len(table_headers[current_table])
                actual_columns = len(row)
                
                if actual_columns != expected_columns:
                    print(f"Warning: Line {line_number} has {actual_columns} columns, expected {expected_columns}. Skipping row.")
                    continue  # Skip rows that don't match the expected column count
                
                # Replace '(Data)' placeholders with actual data if necessary
                cleaned_row = [cell if cell != '(Data)' else None for cell in row]
                data_tables[current_table].append(cleaned_row)
            else:
                # Rows outside of any table headers are ignored
                print(f"Warning: Line {line_number} is outside of any table. Skipping row.")
                continue
    
    # Convert lists to DataFrames with appropriate columns
    df_tables = {}
    for table_key, data in data_tables.items():
        if data:  # Only create DataFrame if there's data
            df = pd.DataFrame(data, columns=table_headers[table_key])
            df_tables[table_key] = df
        else:
            df_tables[table_key] = pd.DataFrame(columns=table_headers[table_key])
    
    return df_tables['table1'], df_tables['table2'], df_tables['table3']



file_path = './Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1920-2.csv'  
df_table1, df_table2, df_table3 = load_export_csv(file_path)

# Display the DataFrames
print("Table 1 DataFrame:")
print(df_table1.head())

print("\nTable 2 DataFrame:")
print(df_table2.head())

print("\nTable 3 DataFrame:")
print(df_table3.head())

# Optionally, save the DataFrames to separate CSV files
df_table1.to_csv(os.path.join('./DataTables', 'table_1.csv'), index=False)
df_table2.to_csv(os.path.join('./DataTables', 'table_2.csv'), index=False)
df_table3.to_csv(os.path.join('./DataTables', 'table_3.csv'), index=False)


Table 1 DataFrame:
  DeviceType SerialNumber Description        EventDateTime Readings (mg/dL)
0    Unknown       870772         EGV  2023-03-01T00:00:55              174
1    Unknown       870772         EGV  2023-03-01T00:05:55              172
2    Unknown       870772         EGV  2023-03-01T00:10:55              171
3    Unknown       870772         EGV  2023-03-01T00:15:55              168
4    Unknown       870772         EGV  2023-03-01T00:20:55              165

Table 2 DataFrame:
  DeviceType SerialNumber Description        EventDateTime BG (mg/dL) Note
0    Unknown       870772          BG  2023-03-01T16:07:59        238     
1    Unknown       870772          BG  2023-03-01T18:04:52        382     
2    Unknown       870772          BG  2023-03-01T20:09:14        156     
3    Unknown       870772          BG  2023-03-01T20:31:14        248     
4    Unknown       870772          BG  2023-03-02T12:46:15        157     

Table 3 DataFrame:
    Type BolusType BolusDeliveryMet

In [2]:
import glob

def gather_csv_files(main_directory):
    """
    Gathers all CSV files from subdirectories (2022PumpData, 2023PumpData, 2024PumpData)
    in the 'data' directory located in the main directory.

    Parameters:
        main_directory (str): The path to the main directory.

    Returns:
        list: A list of paths to all CSV files found in the specified subdirectories.
    """
    # Define the parent directory
    parent_directory = os.path.join(main_directory, 'Data')
    
    # List of subdirectories to search
    subdirectories = ['2022PumpData', '2023PumpData', '2024PumpData']
    
    # Collect CSV files from all subdirectories
    csv_files = []
    for subdir in subdirectories:
        subdir_path = os.path.join(parent_directory, subdir)
        print(subdir_path)
        csv_files.extend(glob.glob(os.path.join(subdir_path, '*.csv')))
    
    return csv_files

# Usage example
main_directory = './'  # Replace with the path to your main directory
csv_files = gather_csv_files(main_directory)

# Print the gathered CSV file paths
for file in csv_files:
    print(file)


./Data/2022PumpData
./Data/2023PumpData
./Data/2024PumpData
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1917-2.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1910.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1912.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1916-2.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1917.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1916.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1914.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1918.csv
./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1908.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1924.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1918.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1923-2.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1922.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1923.csv
./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1921-3.csv
./Data/2023PumpDa

In [3]:

def process_all_csv_files_combined(main_directory, output_directory):
    """
    Processes all CSV files in the 'Data/2022PumpData', 'Data/2023PumpData', and 'Data/2024PumpData'
    subdirectories, extracting tables and appending them into combined CSV files.

    Parameters:
        main_directory (str): The path to the main directory containing the 'Data' folder.
        output_directory (str): The path to the directory where combined tables will be saved.

    Returns:
        None
    """
    # Gather all CSV files
    csv_files = gather_csv_files(main_directory)

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Initialize empty DataFrames for combined output
    combined_table1 = pd.DataFrame()
    combined_table2 = pd.DataFrame()
    combined_table3 = pd.DataFrame()

    for file_path in csv_files:
        print(f"Processing file: {file_path}")
        
        try:
            # Load the CSV file and extract tables
            df_table1, df_table2, df_table3 = load_export_csv(file_path)
            
            # Append each table to its respective combined DataFrame
            combined_table1 = pd.concat([combined_table1, df_table1], ignore_index=True)
            combined_table2 = pd.concat([combined_table2, df_table2], ignore_index=True)
            combined_table3 = pd.concat([combined_table3, df_table3], ignore_index=True)
        
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Define output file paths
    table1_file = os.path.join(output_directory, 'EVG.csv')
    table2_file = os.path.join(output_directory, 'BG.csv')
    table3_file = os.path.join(output_directory, 'Bolus.csv')

    # Save the combined tables to CSV
    combined_table1.to_csv(table1_file, index=False)
    combined_table2.to_csv(table2_file, index=False)
    combined_table3.to_csv(table3_file, index=False)

    print(f"Combined tables saved to: {output_directory}")


# Usage example
main_directory = './'  # Replace with the path to your main directory
output_directory = './DataTables'  # Replace with your desired output directory

process_all_csv_files_combined(main_directory, output_directory)


./Data/2022PumpData
./Data/2023PumpData
./Data/2024PumpData
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1917-2.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1910.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1912.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1916-2.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1917.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1916.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1914.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1918.csv
Processing file: ./Data/2022PumpData/CSV_redacted_90945369_02Dec2024_1908.csv
Processing file: ./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1924.csv
Processing file: ./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1918.csv
Processing file: ./Data/2023PumpData/CSV_redacted_90945369_02Dec2024_1923-2.cs

In [5]:

# Load the CSV files into pandas DataFrames
file_path1 = './DataTables/BG.csv'  # Replace with the actual path for Table 1
file_path2 = './DataTables/EVG.csv'  # Replace with the actual path for Table 2
file_path3 = './DataTables/Bolus.csv'  # Replace with the actual path for Table 3

# Load the data
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)

# Convert the EventDateTime column to datetime and extract dates
df1['EventDate'] = pd.to_datetime(df1['EventDateTime']).dt.date
df2['EventDate'] = pd.to_datetime(df2['EventDateTime']).dt.date
df3['EventDate'] = pd.to_datetime(df3['CompletionDateTime']).dt.date

# Find the unique dates for each table
unique_dates1 = set(df1['EventDate'].unique())
unique_dates2 = set(df2['EventDate'].unique())
unique_dates3 = set(df3['EventDate'].unique())

print(len(unique_dates3))

# Find the intersection of unique dates across all three tables
common_dates = unique_dates1.intersection(unique_dates2).intersection(unique_dates3)

# Print the number of unique intersecting dates
print(f"Number of unique dates that intersect across all three tables: {len(common_dates)}")


663
Number of unique dates that intersect across all three tables: 599
